In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 35% 9.00M/25.7M [00:00<00:00, 89.7MB/s]
100% 25.7M/25.7M [00:00<00:00, 164MB/s] 


In [5]:
import zipfile
zip_ref = zipfile.ZipFile("/content/imdb-dataset-of-50k-movie-reviews.zip","r")
zip_ref.extractall("/content")
zip_ref.close()

In [10]:
import pandas as pd

df = pd.read_csv("/content/imdb-dataset-of-50k-movie-reviews.zip")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
df["review"][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

**( 1.)  LowerCasing**

In [12]:
df["review"][3].lower()

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [13]:
df["review"] = df["review"].str.lower()

In [14]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


 (2.) Remove HTML Tags   

In [15]:
import re
def remove_html_tags(text):
  pattern  = re.compile('<.*?>')
  return pattern.sub(r'',text)

In [18]:
df["review"] = df["review"].apply(remove_html_tags)

In [20]:
df.head()
# All html tags are removed ...

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


(3.) Remove URLs

In [22]:
text = "Check out my blog at https://example.com for more insights. You can also visit my GitHub repository at https://github.com/username/repo. Follow me on Twitter at http://twitter.com/username for updates."


In [25]:
def remove_url(text):
  pattern = re.compile(r'http[s]?://\S+')
  return pattern.sub(r'',text)

In [26]:
remove_url(text)

'Check out my blog at  for more insights. You can also visit my GitHub repository at  Follow me on Twitter at  for updates.'

(4.) Remove Punctuation

In [27]:
import string ,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
exclude = string.punctuation

In [32]:
def remove_punc(text):
  for char in exclude:
    text = text.replace(char,'')
  return text

In [33]:
text = 'string .with , punctuation??!'

In [34]:
remove_punc(text)

'string with  punctuation'

In [38]:
start = time.time()
print(remove_punc(text))
time1 = time.time() - start
print(time1)  # time taking

string with  punctuation
0.00016951560974121094


In [39]:
#  lets make a faster code

def remove_punc1(text):
  return text.translate(str.maketrans('','',exclude))

In [42]:
start = time.time()
print(remove_punc1(text))
time2 = time.time() - start
print(time2)  # time taking

print(time1/time2)

string with  punctuation
0.0010695457458496094
1.1270619705751226


(5.) Chat Word Treatment

In [43]:
# ASAP --> as soon as possible
# gn --> good night

In [44]:
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That Feeling When",
    "MFW": "My Face When",
    "MRW": "My Reaction When",
    "IFYP": "I Feel Your Pain",
    "LOL": "Laughing Out Loud",
    "TNTL": "Trying Not To Laugh",
    "JK": "Just Kidding",
    "IDC": "I Don’t Care",
    "ILY": "I Love You",
    "IMU": "I Miss You",
    "ADIH": "Another Day In Hell",
    "ZZZ": "Sleeping, Bored, Tired",
    "WYWH": "Wish You Were Here",
    "TIME": "Tears In My Eyes",
    "BAE": "Before Anyone Else",
    "FIMH": "Forever In My Heart",
    "BSAAW": "Big Smile And A Wink",
    "BWL": "Bursting With Laughter",
    "LMAO": "Laughing My A.. Off",
    "BFF": "Best Friends Forever",
    "CSL": "Can’t Stop Laughing"
}


In [50]:
def chat_conversion(text):  # Function for conversion of the Short form chat
  new_text = []
  for w in text.split():
    if w.upper() in chat_words:
      new_text.append(chat_words[w.upper()])
    else:
      new_text.append(w)

  return " ".join(new_text)

In [51]:
chat_conversion("IMHO he is the best")

'In My Honest/Humble Opinion he is the best'

(6.)  Spelling Correction

In [54]:
from textblob import  TextBlob

incorrect_text = " ceertain conditionas durring seveal ggenerations aree modified in the same manneer "

textblb  = TextBlob(incorrect_text)

textblb.correct().string

' certain conditions during several generations are modified in the same manner '

(6.) Removing Stop words

In [59]:
# removing stopword is importent for " Sentiment Analysis" and " Document Classification"
#  they are not removed in "POS"

import nltk
from nltk.corpus import stopwords

# Download the stopwords
nltk.download('stopwords')

# Now you can use the stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)


{'if', "hasn't", "isn't", "you'd", 'that', 'were', 'down', 'wouldn', 'theirs', 'themselves', 'such', 'is', 'was', 'weren', 'for', "didn't", 'having', 'm', 'himself', "don't", 'on', 'am', 'when', 'hasn', "shan't", "that'll", 'they', 'against', 'has', 'be', 'other', "you're", 'we', "couldn't", 'yourself', 'these', 'shan', 'his', 'or', 'shouldn', 'won', 'd', 'ma', 'further', "shouldn't", 'above', 'who', 'to', 'over', 'so', 'doesn', "you've", 'just', 'more', 'as', "mightn't", 'an', 'hadn', "wouldn't", 'them', 'being', 'all', 'll', 'doing', 's', 're', 'and', 'i', 'here', 'your', 'ain', 'needn', 'it', 'because', 'why', 'herself', 'her', 'she', 'o', 'how', 'own', 'most', 'under', 'than', 'mustn', 'had', 'where', 'into', 'its', 'after', 'while', 'me', "weren't", "you'll", "won't", 've', 'through', 'only', 'about', 'before', 'whom', 'yours', 'few', 'a', 'not', 'too', 'do', 'from', 'hers', "hadn't", 'again', 'will', 'my', 'in', 'y', 'him', 'which', 'both', 'wasn', 'of', "mustn't", 'you', 'are', 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
def remove_stopword(text):
  new_text = []

  for word in text.split():
    if word in stop_words:
      new_text.append("")
    else:
      new_text.append(word)

  x  = new_text[:]
  new_text.clear()
  return " ".join(x)

In [61]:
remove_stopword("basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time")

"basically there's  family   little boy (jake) thinks there's  zombie   closet &  parents  fighting   time"

In [62]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [63]:
df["review"].apply(remove_stopword)

,review
0,one reviewers mentioned watching 1 oz e...
1,wonderful little production. filming techniq...
2,thought wonderful way spend time hot s...
3,basically there's family little boy (jake) ...
4,"petter mattei's ""love time money"" visuall..."
...,...
49995,thought movie right good job. creative...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schoo...
49998,i'm going disagree previous comment side...


(7.) Remove Emoji's ---------------------------------------------------------

In [64]:
import re

def remove_emojis(text):
    # Emoji pattern
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # Chinese characters
        "\U00002702-\U000027B0"  # Dingbats
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "\U0001F926-\U0001F937"  # Supplemental symbols and pictographs
        "\U00010000-\U0010FFFF"  # Supplementary Private Use Area-A
        "\u200d"                 # Zero-width joiner
        "\u2640-\u2642"          # Gender symbols
        "\u2600-\u2B55"          # Miscellaneous symbols
        "\u23cf"                 # Eject button
        "\u23e9"                 # Fast forward
        "\u231a"                 # Watch
        "\ufe0f"                 # Dingbats
        "\u3030"                 # Wavy dash
        "]+",
        flags=re.UNICODE
    )

    return emoji_pattern.sub(r'', text)

# Example usage
text_with_emojis = "Hello 😊! How are you? 🤔 Let's grab a coffee ☕️."
clean_text = remove_emojis(text_with_emojis)
print(clean_text)


Hello ! How are you?  Let's grab a coffee .


In [70]:
#  To replace emoji with there meaning ..

!pip install emoji


In [69]:
import emoji
emoji.demojize('Python is 🔥!')

'Python is :fire:!'

(7.)  Tokenization ---------------------------------------------

spaCy Lib   **(BEST)**

In [72]:
!pip install spacy
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [73]:

import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Example text
text = "Hello world! This is a test sentence."

# Tokenize text
doc = nlp(text)

# Extract tokens
tokens = [token.text for token in doc]
print("Tokens:", tokens)

Tokens: ['Hello', 'world', '!', 'This', 'is', 'a', 'test', 'sentence', '.']


nltk Lib

In [74]:
!pip install nltk



In [75]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize

# Example text
text = "Hello world! This is a test sentence."

# Tokenize into words
words = word_tokenize(text)
print("Word tokens:", words)

# Tokenize into sentences
sentences = sent_tokenize(text)
print("Sentence tokens:", sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Word tokens: ['Hello', 'world', '!', 'This', 'is', 'a', 'test', 'sentence', '.']
Sentence tokens: ['Hello world!', 'This is a test sentence.']


(8.) Stemming

In [76]:
import nltk
from nltk.stem import PorterStemmer, LancasterStemmer

# Download necessary resources
nltk.download('punkt')

# Initialize stemmers
porter = PorterStemmer()
# lancaster = LancasterStemmer()

# Example text
text = "running runs runner easily running"

# Tokenize the text
words = nltk.word_tokenize(text)

# Apply stemming
porter_stemmed = [porter.stem(word) for word in words]
# lancaster_stemmed = [lancaster.stem(word) for word in words]

print("Porter Stemmed:", porter_stemmed)
# print("Lancaster Stemmed:", lancaster_stemmed)


Porter Stemmed: ['run', 'run', 'runner', 'easili', 'run']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Lemmetization (slow) -> used if output is to be shown to user.

In [77]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Example text
text = "running runs runner easily running"

# Process the text
doc = nlp(text)

# Extract lemmatized tokens
lemmatized = [token.lemma_ for token in doc]
print("Lemmatized:", lemmatized)


Lemmatized: ['run', 'run', 'runner', 'easily', 'run']
